# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print(f'State: {states}')

In [ ]:
len(states)
print(type(states))

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import time

env = UnityEnvironment(file_name='Reacher.app')
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [2]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [3]:
from ddpg_agent import Agent
agent = Agent(state_size=state_size, action_size=action_size, random_seed=42)

def ddpg(num_agents=20, action_size=action_size, n_episodes=200, max_t=1000):
    """Deep Deterministic Policy Gradient"""
    scores = []
    scores_deque = deque(maxlen=100)
    step_count = 0
    for episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        agent.reset() # reset the noise of the agent
        eps_scores = np.zeros(num_agents)
        for t in range(max_t):
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            agent.step(states, actions, rewards, next_states, dones, step_count)
            states = next_states
            eps_scores += rewards
            step_count += 1
            if np.any(dones):
                break
        
        scores_deque.append(np.mean(eps_scores))
        scores.append(np.mean(eps_scores))
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if episode % 10 == 0:
            print(f'Episode {episode} \tAverage Score {np.mean(scores_deque):.2f} \tMax Score {np.max(scores_deque):.2f}')
    
    print(f'Episode {episode} \tAverage Score {np.mean(scores_deque):.2f}')
    return scores

start = time.time()
scores = ddpg()
print(f'Time elapsed: {(time.time()-start)/360:.3f} hours')

Episode 10 	Average Score 0.92 	Max Score 1.43
Episode 20 	Average Score 0.53 	Max Score 1.43
Episode 30 	Average Score 0.65 	Max Score 1.86
Episode 40 	Average Score 0.94 	Max Score 2.82
Episode 50 	Average Score 1.64 	Max Score 6.64
Episode 60 	Average Score 2.62 	Max Score 9.69
Episode 70 	Average Score 3.86 	Max Score 13.22
Episode 80 	Average Score 5.16 	Max Score 15.63
Episode 90 	Average Score 6.43 	Max Score 17.76
Episode 100 	Average Score 8.09 	Max Score 26.66
Episode 110 	Average Score 10.89 	Max Score 31.76
Episode 120 	Average Score 14.32 	Max Score 36.33
Episode 130 	Average Score 17.78 	Max Score 37.48
Episode 140 	Average Score 21.22 	Max Score 37.48
Episode 150 	Average Score 24.38 	Max Score 37.48
Episode 160 	Average Score 27.19 	Max Score 37.48
Episode 170 	Average Score 29.60 	Max Score 37.48
Episode 180 	Average Score 31.67 	Max Score 37.48
Episode 190 	Average Score 33.53 	Max Score 37.48
Episode 200 	Average Score 34.93 	Max Score 38.59
Episode 200 	Average Scor

In [5]:
env.close()

In [9]:
plt.plot(scores)
plt.savefig('media/ddpg_exp4.png')

## A few experiments

### First successfully trained agent

- 128x128x4 network 
- normal(0,128^-0.5) initialization
- batch size: 1024
- buffer size: 1e5 
- actor learning rate: 1e-4 
- critic learning rate: 1e-3 
- network updates every: 1 step
- number of training episodes: 130
- max steps per episode: 1000

This took about 2 hours to train to completion. Technically not solved, but was definitely close at the 130 episode mark (should have trained longer).

![](media/ddpg_original.png)

### Second attempt

- 128x128x4 network 
- normal(0,128^-0.5) initialization
- batch size: 256
- buffer size: 1e5 
- actor learning rate: 1e-4 
- critic learning rate: 1e-3 
- network updates every: 10 steps
- number of training episodes: 150
- max steps per episode: 1000

This only took about 20 minutes, thanks to the decreased batch size and fewer network updates, but had terrible performance.

![](media/ddpg_exp2.png)

### Third attempt

- 128x128x4 network 
- normal(0,128^-0.5) initialization
- batch size: 512
- buffer size: 1e6 
- actor learning rate: 1e-4 
- critic learning rate: 1e-3 
- network updates every: 1 step
- number of training episodes: 200
- max steps per episode: 1000

![](media/ddpg_exp3.png)

### Fourth attempt

- 128x128x4 network 
- xavier_normal initialization
- batch size: 512
- buffer size: 1e5 
- actor learning rate: 1e-3 
- critic learning rate: 1e-3 
- network updates every: 1 step
- number of training episodes: 200
- max steps per episode: 1000

A few notes. This agent trained pretty well. I previously played around with a few different setups and found these hyperparameters were training decently well but was relatively slow and the scores were increasing even after 200 episodes. So I increased the actor learning rate by 10 and that did the trick. Python says this took 19 hours to train but there is no way that is true. My math must be off :).

![](media/ddpg_exp4.png)